<a href="https://colab.research.google.com/github/brianxiadong/LLMs-from-scratch-CN-Plus/blob/main/ch03/01_main-chapter-code/ch03.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

<table style="width:100%">
<tr>
<td style="vertical-align:middle; text-align:left;">
<font size="2">
Supplementary code for the <a href="http://mng.bz/orYv">Build a Large Language Model From Scratch</a> book by <a href="https://sebastianraschka.com">Sebastian Raschka</a><br>
<br>Code repository: <a href="https://github.com/rasbt/LLMs-from-scratch">https://github.com/rasbt/LLMs-from-scratch</a>
</font>
</td>
<td style="vertical-align:middle; text-align:left;">
<a href="http://mng.bz/orYv"><img src="https://sebastianraschka.com/images/LLMs-from-scratch-images/cover-small.webp" width="100px"></a>
</td>
</tr>
</table>


# 第三章：从零开始实现注意力机制

本notebook中使用的包：

In [1]:
from importlib.metadata import version

print("torch version:", version("torch"))

torch version: 2.6.0+cu124


本章主要介绍LLM中的引擎，注意力机制。我们会从一个简单的注意力机制开始，逐步进行完善。

<img src="https://sebastianraschka.com/images/LLMs-from-scratch-images/ch03_compressed/01.webp?123" width="500px">

<img src="https://sebastianraschka.com/images/LLMs-from-scratch-images/ch03_compressed/02.webp" width="600px">

## 3.1 长文本建模中常见的问题


- 本节不包含代码
- 由于在翻译中，源语言和目标语言在语法结构上存在差异，所以逐字翻译是不合理的：
从下图可以看到德文和英文在同一个句子上的语法顺序是不一样的。

<img src="https://sebastianraschka.com/images/LLMs-from-scratch-images/ch03_compressed/03.webp" width="400px">

- 在 Transformer 模型引入之前，一般使用编码器-解码器的循环神经网络RNN进行机器翻译任务
- 在这个架构中，使用编码器处理源语言的一系列token，使用一个隐藏状态（理解为神经网络中的一种中间层）。同样的解码器也通过一层隐藏状态去输出结果。

<img src="https://sebastianraschka.com/images/LLMs-from-scratch-images/ch03_compressed/04.webp" width="500px">

## 3.2 使用注意力机制去捕获数据之间的依赖关系


- 本节不包含代码
- 通过注意力机制，神经网络中的文本生成解码器部分能够选择性地访问所有输入token，这意味着在生成特定输出token时，某些输入token比其他token更重要：
> 这是实现上下文关系的核心思想


<img src="https://sebastianraschka.com/images/LLMs-from-scratch-images/ch03_compressed/05.webp" width="500px">

- Transformer 中的自注意力机制是一种通过使句子中的每个位置都能够参与，并确定同一句子中每个其他位置的相关性来增强输入表示的技术

> 当处理一个句子（比如 “小明在公园和小红一起放风筝”）时，句子里的每个词（每个位置）都不会孤立存在，而是会主动 “关注” 句子中其他所有词，判断自己和它们的关联程度（比如 “小明” 和 “小红”“一起” 的相关性更高，和 “风筝” 的相关性稍低）。
通过这种方式，每个词的最终表示（可以理解为模型对这个词的 “理解”）不仅包含了自身的含义，还融合了那些和它关系密切的词的信息。比如 “一起” 这个词的表示，会带上 “小明”“小红” 的信息，让模型更清楚 “一起” 是指谁和谁一起。
这样一来，句子的输入表示就从孤立的词变成了带有上下文关联的、更丰富的信息，从而让模型能更准确地理解句子的整体意思 —— 这就是 “增强输入表示” 的作用。

<img src="https://sebastianraschka.com/images/LLMs-from-scratch-images/ch03_compressed/06.webp" width="300px">

## 3.3 使用自注意力机制关注输入的不同部分

### 3.3.1 实现一个没有可训练权重的简单自注意力机制

- 本节解释了一种非常简化的自注意力实现，其中不包含任何可训练的权重
- 并不是 transformers 中使用的注意力机制，只是用来理解注意力机制
- 在3.3.2 节中将扩展这个简单的注意力机制来实现真正的自注意力机制
- 假设我们获得一个输出 $x^{(1)}$ to $x^{(T)}$
  - 输入是一个文本（例如，像"你的旅程始于一步"这样的句子），它已经按照第 2 章所述转换成了词嵌入
  - 例如, $x^{(1)}$ 是一个 d 维向量，表示单词 "Your"，等等
- **目标:** 计算 $x^{(1)}$ 到 $x^{(T)}$ 中每个输入序列元素 $x^{(i)}$ 的上下文向量 $z^{(i)}$（其中 $z$ 和 $x$ 具有相同的维度）
    - 上下文向量是对输入的加权求和 𝑥(𝑇)

> 咱们可以这么理解：
假设有一串输入，比如 “我今天去公园玩” 里的每个词，咱们给它们编号，第一个词是 x (1)，第二个是 x (2)，一直到最后一个 x (T)。
现在要给每个词算一个 “上下文向量” z (i)，这个 z (i) 和原来的词 x (i) 长度一样，但它是 “结合了上下文” 的结果。
拿第二个词 x (2) 来说，它对应的上下文向量 z (2)，不是只看 x (2) 自己，而是把所有词（从 x (1) 到 x (T)）都加起来，但每个词加多少，是由它和 x (2) 的 “关联度” 决定的 —— 这就是 “注意力权重”。关联度高的词，在总和里占比就大，贡献就多；关联度低的，占比就小。
所以简单说，z (2) 就像是 x (2) 的 “升级版”：它保留了 x (2) 本身的信息，还融合了其他和当前任务相关的词的信息，让 x (2) 更贴合具体场景的需求。


<img src="https://sebastianraschka.com/images/LLMs-from-scratch-images/ch03_compressed/07.webp" width="400px">

- （请注意，此图中的数字小数点后只保留了一位是为了更容易看清；类似地，其他图形也可能包含截断值）

- 未归一化的注意力权重被称为"注意力分数"，而归一化后的注意力分数（总和为 1），则被称为"注意力权重"


- 下面的代码逐步解释了上面的图示 ：

<br>

- 步骤 1：计算未归一化的注意力分数 $\omega$
- 假设我们使用第二个输入词作为查询，即 $q^{(2)} = x^{(2)}$，我们通过点积计算未归一化的注意力分数：
   - 𝜔21=𝑥(1)𝑞(2)⊤
   - 𝜔22=𝑥(2)𝑞(2)⊤
   - 𝜔23=𝑥(3)𝑞(2)⊤
   - ...
   - 𝜔2𝑇=𝑥(𝑇)𝑞(2)⊤

- $\omega$ 是希腊字母 "omega"，用于表示未归一化的注意力分数
    - 下标 "2 1" 表示输入序列中的第 2 个元素被用作查询，针对输入序列中的第 1 个元素

- 假设我们有以下输入句子，它已经被嵌入为三维向量，如第三章所述（为了便于说明，我们使用了非常小的嵌入维度，这样它可以在页面上一行显示而不会换行）：

In [2]:
import torch

inputs = torch.tensor(
  [[0.43, 0.15, 0.89], # Your     (x^1)
   [0.55, 0.87, 0.66], # journey  (x^2)
   [0.57, 0.85, 0.64], # starts   (x^3)
   [0.22, 0.58, 0.33], # with     (x^4)
   [0.77, 0.25, 0.10], # one      (x^5)
   [0.05, 0.80, 0.55]] # step     (x^6)
)

- （在本书中，我们遵循机器学习和深度学习中的常见惯例，其中训练样本表示为行，特征值表示为列；在上面的张量中，每一行代表一个词，每一列代表一个嵌入维度）

- 本节的主要目标是演示如何使用第二个输入序列 $x^{(2)}$ 作为查询来计算上下文向量 $z^{(2)}$

- 该图描绘了这一过程的初始步骤，涉及通过点积操作计算 $x^{(2)}$ 与所有其他输入元素之间的注意力分数 ω :

<img src="https://sebastianraschka.com/images/LLMs-from-scratch-images/ch03_compressed/08.webp" width="400px">

- 我们使用输入序列元素 2，$x^{(2)}$，作为示例来计算上下文向量$z^{(2)}$；在本节稍后，我们将推广这一方法来计算所有上下文向量。
- 第一步是计算未归一化的注意力分数，方法是通过计算查询$x^{(2)}$与所有其他输入标记之间的点积：

In [39]:
query = inputs[1]  # 2nd input token is the query

attn_scores_2 = torch.empty(inputs.shape[0])
for i, x_i in enumerate(inputs):
    attn_scores_2[i] = torch.dot(x_i, query) # dot product (transpose not necessary here since they are 1-dim vectors)

print(attn_scores_2)

tensor([0.9544, 1.4950, 1.4754, 0.8434, 0.7070, 1.0865])


- 点积本质上是逐元素相乘并求和结果的简写：

In [40]:
res = 0.

for idx, element in enumerate(inputs[0]):
    res += inputs[0][idx] * query[idx]

print(res)
print(torch.dot(inputs[0], query))

tensor(0.9544)
tensor(0.9544)


- 步骤 2：将未归一化的注意力分数（"omegas"，$\omega$）进行归一化处理，使其总和为 1
- 这里有一种简单的方法来归一化未归一化的注意力分数，使其总和为 1（一种惯例，便于解释，且对训练稳定性很重要）：

<img src="https://sebastianraschka.com/images/LLMs-from-scratch-images/ch03_compressed/09.webp" width="500px">

In [5]:
attn_weights_2_tmp = attn_scores_2 / attn_scores_2.sum()

print("Attention weights:", attn_weights_2_tmp)
print("Sum:", attn_weights_2_tmp.sum())

Attention weights: tensor([0.1455, 0.2278, 0.2249, 0.1285, 0.1077, 0.1656])
Sum: tensor(1.0000)


- 然而，在实际应用中，通常推荐使用 softmax 函数进行归一化，因为 softmax 函数在处理极端值方面表现更好，并且在训练过程中具有更理想的梯度特性。
- 这里是一个用于缩放的 softmax 函数的朴素实现，它同时将向量元素归一化，使其总和为 1：


In [6]:
def softmax_naive(x):
    return torch.exp(x) / torch.exp(x).sum(dim=0)

attn_weights_2_naive = softmax_naive(attn_scores_2)

print("Attention weights:", attn_weights_2_naive)
print("Sum:", attn_weights_2_naive.sum())

Attention weights: tensor([0.1385, 0.2379, 0.2333, 0.1240, 0.1082, 0.1581])
Sum: tensor(1.)


- 上述朴素实现由于溢出和下溢问题，在处理大值或小值输入时可能会出现数值不稳定性问题
- 因此，在实际应用中，建议使用经过高度优化的 PyTorch 实现 softmax：

In [7]:
attn_weights_2 = torch.softmax(attn_scores_2, dim=0)

print("Attention weights:", attn_weights_2)
print("Sum:", attn_weights_2.sum())

Attention weights: tensor([0.1385, 0.2379, 0.2333, 0.1240, 0.1082, 0.1581])
Sum: tensor(1.)


- 步骤 3：通过将嵌入的输入标记 $x^{(i)}$ 与注意力权重相乘，并求和得到的向量来计算上下文向量 $z^{(2)}$：

<img src="https://sebastianraschka.com/images/LLMs-from-scratch-images/ch03_compressed/10.webp" width="500px">

In [8]:
query = inputs[1] # 2nd input token is the query

context_vec_2 = torch.zeros(query.shape)
for i,x_i in enumerate(inputs):
    context_vec_2 += attn_weights_2[i]*x_i

print(context_vec_2)

tensor([0.4419, 0.6515, 0.5683])


### 3.3.2 计算所有输入 token 的注意力权重

#### 推广到所有输入序列 token：

- 前面我们计算了输入 2 的注意力权重和上下文向量（如图下方高亮行所示）
- 接下来我们将推广这一计算，以计算所有注意力权重和上下文向量

<img src="https://sebastianraschka.com/images/LLMs-from-scratch-images/ch03_compressed/11.webp" width="400px">

- （请注意，此图中数字的小数点后保留两位以减少视觉杂乱；每行中的数值应相加等于 1.0 或 100%；类似地，其他图中的数字也进行了截断）

- 在自注意力机制中，过程始于注意力分数的计算，随后将注意力分数进行归一化以得到总和为 1 的注意力权重
- 这些注意力权重随后被用于通过输入的加权求和来生成上下文向量

<img src="https://sebastianraschka.com/images/LLMs-from-scratch-images/ch03_compressed/12.webp" width="400px">

- Apply previous **step 1** to all pairwise elements to compute the unnormalized attention score matrix:

In [9]:
attn_scores = torch.empty(6, 6)

for i, x_i in enumerate(inputs):
    for j, x_j in enumerate(inputs):
        attn_scores[i, j] = torch.dot(x_i, x_j)

print(attn_scores)

tensor([[0.9995, 0.9544, 0.9422, 0.4753, 0.4576, 0.6310],
        [0.9544, 1.4950, 1.4754, 0.8434, 0.7070, 1.0865],
        [0.9422, 1.4754, 1.4570, 0.8296, 0.7154, 1.0605],
        [0.4753, 0.8434, 0.8296, 0.4937, 0.3474, 0.6565],
        [0.4576, 0.7070, 0.7154, 0.3474, 0.6654, 0.2935],
        [0.6310, 1.0865, 1.0605, 0.6565, 0.2935, 0.9450]])


- 我们可以通过矩阵乘法更高效地实现上述功能：

In [10]:
attn_scores = inputs @ inputs.T
print(attn_scores)

tensor([[0.9995, 0.9544, 0.9422, 0.4753, 0.4576, 0.6310],
        [0.9544, 1.4950, 1.4754, 0.8434, 0.7070, 1.0865],
        [0.9422, 1.4754, 1.4570, 0.8296, 0.7154, 1.0605],
        [0.4753, 0.8434, 0.8296, 0.4937, 0.3474, 0.6565],
        [0.4576, 0.7070, 0.7154, 0.3474, 0.6654, 0.2935],
        [0.6310, 1.0865, 1.0605, 0.6565, 0.2935, 0.9450]])


- 与之前的步骤 2 类似，我们将每一行进行归一化处理，使得每一行的值之和为 1：

In [11]:
attn_weights = torch.softmax(attn_scores, dim=-1)
print(attn_weights)

tensor([[0.2098, 0.2006, 0.1981, 0.1242, 0.1220, 0.1452],
        [0.1385, 0.2379, 0.2333, 0.1240, 0.1082, 0.1581],
        [0.1390, 0.2369, 0.2326, 0.1242, 0.1108, 0.1565],
        [0.1435, 0.2074, 0.2046, 0.1462, 0.1263, 0.1720],
        [0.1526, 0.1958, 0.1975, 0.1367, 0.1879, 0.1295],
        [0.1385, 0.2184, 0.2128, 0.1420, 0.0988, 0.1896]])


- 快速验证每一行的值之和确实为 1：

In [12]:
row_2_sum = sum([0.1385, 0.2379, 0.2333, 0.1240, 0.1082, 0.1581])
print("Row 2 sum:", row_2_sum)

print("All row sums:", attn_weights.sum(dim=-1))

Row 2 sum: 1.0
All row sums: tensor([1.0000, 1.0000, 1.0000, 1.0000, 1.0000, 1.0000])


- 应用之前的步骤 3 来计算所有上下文向量：

In [13]:
all_context_vecs = attn_weights @ inputs
print(all_context_vecs)

tensor([[0.4421, 0.5931, 0.5790],
        [0.4419, 0.6515, 0.5683],
        [0.4431, 0.6496, 0.5671],
        [0.4304, 0.6298, 0.5510],
        [0.4671, 0.5910, 0.5266],
        [0.4177, 0.6503, 0.5645]])


- 先前计算出的上下文向量 $z^{(2)} = [0.4419, 0.6515, 0.5683]$ 可以在上面的第 2 行找到：

In [14]:
print("Previous 2nd context vector:", context_vec_2)

Previous 2nd context vector: tensor([0.4419, 0.6515, 0.5683])


## 3.4 使用可训练权重实现自注意力机制

- 这是一个概念性的框架，用来解释本节所讲的自注意力机制，是如何与本书及本章的整体内容和结构相衔接的。


<img src="https://sebastianraschka.com/images/LLMs-from-scratch-images/ch03_compressed/13.webp" width="400px">

### 3.4.1 逐步计算注意力权重

- 在本节中，我们正在实现原始 Transformer 架构、GPT 模型以及大多数其他流行 LLMs 中使用的自我注意力机制
- 这种自我注意力机制也称为"缩放点积注意力"
- 整体思路与之前相似：
  - 我们希望将上下文向量计算为针对特定输入元素的输入向量的加权求和
  - 为此，我们需要注意力权重
- 与之前介绍的基本注意力机制相比，只有细微的差别：
  - 最显著的区别是引入了在模型训练过程中更新的权重矩阵
  - 这些可训练的权重矩阵至关重要，以便模型（特别是模型内部的注意力模块）能够学会生成"良好"的上下文向量

<img src="https://sebastianraschka.com/images/LLMs-from-scratch-images/ch03_compressed/14.webp" width="600px">

- 逐步实现自注意力机制，我们将从介绍三个训练权重矩阵 $W_q$，$W_k$ 和 $W_v$ 开始 𝑊𝑣
- 这三个矩阵用于通过矩阵乘法将嵌入输入标记 $x^{(i)}$ 投影到查询、键和值向量上：

  - 查询向量：$q^{(i)} = x^{(i)}\,W_q $ 𝑞(𝑖)=𝑥(𝑖)𝑊𝑞
  - 键向量：$k^{(i)} = x^{(i)}\,W_k $ 𝑘(𝑖)=𝑥(𝑖)𝑊𝑘
  - 值向量：$v^{(i)} = x^{(i)}\,W_v $ 𝑣(𝑖)=𝑥(𝑖)𝑊𝑣


- 输入 $x$ 的嵌入维度和查询向量 $q$ 的嵌入维度可以相同或不同，这取决于模型的设计和具体实现
- 在 GPT 模型中，输入和输出维度通常是相同的，但为了说明目的，为了更好地遵循计算过程，我们在这里选择不同的输入和输出维度：

In [15]:
x_2 = inputs[1] # second input element
d_in = inputs.shape[1] # the input embedding size, d=3
d_out = 2 # the output embedding size, d=2

- 下面，我们初始化三个权重矩阵；请注意，我们设置 requires_grad=False 是为了减少输出的杂乱，以说明目的，但如果我们要使用权重矩阵进行模型训练，我们会设置 requires_grad=True 来在模型训练过程中更新这些矩阵

In [16]:
torch.manual_seed(123)

W_query = torch.nn.Parameter(torch.rand(d_in, d_out), requires_grad=False)
W_key   = torch.nn.Parameter(torch.rand(d_in, d_out), requires_grad=False)
W_value = torch.nn.Parameter(torch.rand(d_in, d_out), requires_grad=False)

- 接下来我们计算查询、键和值向量：

In [41]:
query_2 = x_2 @ W_query # _2 because it's with respect to the 2nd input element
key_2 = x_2 @ W_key
value_2 = x_2 @ W_value

print(query_2)

tensor([0.4306, 1.4551])


- 如我们所见，我们成功地将 6 个输入 token 从 3D 空间投影到 2D 嵌入空间：

In [42]:
keys = inputs @ W_key
values = inputs @ W_value

print("keys.shape:", keys.shape)
print("values.shape:", values.shape)

keys.shape: torch.Size([6, 2])
values.shape: torch.Size([6, 2])


- 在步骤 2，我们通过计算查询向量和每个键向量之间的点积来计算未归一化的注意力分数：

<img src="https://sebastianraschka.com/images/LLMs-from-scratch-images/ch03_compressed/15.webp" width="600px">

In [43]:
keys_2 = keys[1] # Python starts index at 0
attn_score_22 = query_2.dot(keys_2)
print(attn_score_22)

tensor(1.8524)


- 由于我们有 6 个输入，因此对于给定的查询向量，我们有 6 个注意力分数：

In [20]:
attn_scores_2 = query_2 @ keys.T # All attention scores for given query
print(attn_scores_2)

tensor([1.2705, 1.8524, 1.8111, 1.0795, 0.5577, 1.5440])


<img src="https://sebastianraschka.com/images/LLMs-from-scratch-images/ch03_compressed/16.webp" width="600px">

- 在步骤 3，我们使用之前使用的 softmax 函数来计算注意力权重（归一化的注意力分数，其总和为 1）：
- 与早期版本的不同之处在于，我们现在通过将注意力分数除以嵌入维度的平方根 $\sqrt{d_k}$（即 d_k**0.5 ）来进行缩放：

In [21]:
d_k = keys.shape[1]
attn_weights_2 = torch.softmax(attn_scores_2 / d_k**0.5, dim=-1)
print(attn_weights_2)

tensor([0.1500, 0.2264, 0.2199, 0.1311, 0.0906, 0.1820])


<img src="https://sebastianraschka.com/images/LLMs-from-scratch-images/ch03_compressed/17.webp" width="600px">

- 在步骤 4，我们现在计算输入查询向量 2 的上下文向量：

In [44]:
context_vec_2 = attn_weights_2 @ values
print(context_vec_2)

tensor([0.3061, 0.8210])


### 3.4.2 实现一个紧凑的 SelfAttention 类

- 将所有内容整合起来，我们可以按照以下方式实现自注意力机制：

In [23]:
import torch.nn as nn

class SelfAttention_v1(nn.Module):

    def __init__(self, d_in, d_out):
        super().__init__()
        self.W_query = nn.Parameter(torch.rand(d_in, d_out))
        self.W_key   = nn.Parameter(torch.rand(d_in, d_out))
        self.W_value = nn.Parameter(torch.rand(d_in, d_out))

    def forward(self, x):
        keys = x @ self.W_key
        queries = x @ self.W_query
        values = x @ self.W_value

        attn_scores = queries @ keys.T # omega
        attn_weights = torch.softmax(
            attn_scores / keys.shape[-1]**0.5, dim=-1
        )

        context_vec = attn_weights @ values
        return context_vec

torch.manual_seed(123)
sa_v1 = SelfAttention_v1(d_in, d_out)
print(sa_v1(inputs))

tensor([[0.2996, 0.8053],
        [0.3061, 0.8210],
        [0.3058, 0.8203],
        [0.2948, 0.7939],
        [0.2927, 0.7891],
        [0.2990, 0.8040]], grad_fn=<MmBackward0>)


<img src="https://sebastianraschka.com/images/LLMs-from-scratch-images/ch03_compressed/18.webp" width="400px">

- 我们可以使用 PyTorch 的 Linear 线性层来简化上述实现，如果我们禁用偏差单元，这些层相当于矩阵乘法
- 使用 nn.Linear 而不是我们手动 nn.Parameter(torch.rand(...) 方法的另一个主要优势是 nn.Linear 具有首选的权重初始化方案，这导致模型训练更加稳定

In [24]:
class SelfAttention_v2(nn.Module):

    def __init__(self, d_in, d_out, qkv_bias=False):
        super().__init__()
        self.W_query = nn.Linear(d_in, d_out, bias=qkv_bias)
        self.W_key   = nn.Linear(d_in, d_out, bias=qkv_bias)
        self.W_value = nn.Linear(d_in, d_out, bias=qkv_bias)

    def forward(self, x):
        keys = self.W_key(x)
        queries = self.W_query(x)
        values = self.W_value(x)

        attn_scores = queries @ keys.T
        attn_weights = torch.softmax(attn_scores / keys.shape[-1]**0.5, dim=-1)

        context_vec = attn_weights @ values
        return context_vec

torch.manual_seed(789)
sa_v2 = SelfAttention_v2(d_in, d_out)
print(sa_v2(inputs))

tensor([[-0.0739,  0.0713],
        [-0.0748,  0.0703],
        [-0.0749,  0.0702],
        [-0.0760,  0.0685],
        [-0.0763,  0.0679],
        [-0.0754,  0.0693]], grad_fn=<MmBackward0>)


- 注意到SelfAttention_v1 和 SelfAttention_v2 给出不同的输出，因为它们使用不同的权重矩阵初始值

## 3.5 隐藏未来单词的因果注意力

- 在因果注意力中，对角线以上的注意力权重被屏蔽，确保对于任何给定的输入，LLM 在计算上下文向量时无法利用未来的 token

<img src="https://sebastianraschka.com/images/LLMs-from-scratch-images/ch03_compressed/19.webp" width="400px">

### 3.5.1 实现因果注意力掩码

- 在本节中，我们将之前的自注意力机制转换为因果自注意力机制
- 因果自注意力机制确保模型对序列中某个位置的预测仅依赖于之前已知位置的输出，而不是未来的位置
- 用更简单的话来说，这确保了每个下一个词的预测只应该依赖于前面的词
- 为了实现这一点，对于每个给定的标记，我们将未来的标记（输入文本中当前标记之后的标记）遮蔽掉：

<img src="https://sebastianraschka.com/images/LLMs-from-scratch-images/ch03_compressed/20.webp" width="600px">

- 为了说明和实现因果自注意力机制，让我们使用上一节中的注意力分数和权重：

In [25]:
# Reuse the query and key weight matrices of the
# SelfAttention_v2 object from the previous section for convenience
queries = sa_v2.W_query(inputs)
keys = sa_v2.W_key(inputs)
attn_scores = queries @ keys.T

attn_weights = torch.softmax(attn_scores / keys.shape[-1]**0.5, dim=-1)
print(attn_weights)

tensor([[0.1921, 0.1646, 0.1652, 0.1550, 0.1721, 0.1510],
        [0.2041, 0.1659, 0.1662, 0.1496, 0.1665, 0.1477],
        [0.2036, 0.1659, 0.1662, 0.1498, 0.1664, 0.1480],
        [0.1869, 0.1667, 0.1668, 0.1571, 0.1661, 0.1564],
        [0.1830, 0.1669, 0.1670, 0.1588, 0.1658, 0.1585],
        [0.1935, 0.1663, 0.1666, 0.1542, 0.1666, 0.1529]],
       grad_fn=<SoftmaxBackward0>)


- 屏蔽未来注意力权重的最简单方法是通过 PyTorch 的 tril 函数创建一个掩码，将主对角线以下（包括主对角线本身）的元素设置为 1，将主对角线以上的元素设置为 0：

In [26]:
context_length = attn_scores.shape[0]
mask_simple = torch.tril(torch.ones(context_length, context_length))
print(mask_simple)

tensor([[1., 0., 0., 0., 0., 0.],
        [1., 1., 0., 0., 0., 0.],
        [1., 1., 1., 0., 0., 0.],
        [1., 1., 1., 1., 0., 0.],
        [1., 1., 1., 1., 1., 0.],
        [1., 1., 1., 1., 1., 1.]])


- 然后，我们可以将注意力权重与这个掩码相乘，以将对角线以上的注意力分数置零：

In [27]:
masked_simple = attn_weights*mask_simple
print(masked_simple)

tensor([[0.1921, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.2041, 0.1659, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.2036, 0.1659, 0.1662, 0.0000, 0.0000, 0.0000],
        [0.1869, 0.1667, 0.1668, 0.1571, 0.0000, 0.0000],
        [0.1830, 0.1669, 0.1670, 0.1588, 0.1658, 0.0000],
        [0.1935, 0.1663, 0.1666, 0.1542, 0.1666, 0.1529]],
       grad_fn=<MulBackward0>)


- 然而如果掩码在 softmax 之后应用，就像上面那样，它会破坏 softmax 创建的概率分布
- softmax 确保所有输出值之和为 1
- softmax 之后的掩码会需要重新归一化输出，使其再次之和为 1，这会使过程复杂化，并可能导致意想不到的效果

- 为了确保行之和为 1，我们可以按以下方式归一化注意力权重：

In [28]:
row_sums = masked_simple.sum(dim=-1, keepdim=True)
masked_simple_norm = masked_simple / row_sums
print(masked_simple_norm)

tensor([[1.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.5517, 0.4483, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.3800, 0.3097, 0.3103, 0.0000, 0.0000, 0.0000],
        [0.2758, 0.2460, 0.2462, 0.2319, 0.0000, 0.0000],
        [0.2175, 0.1983, 0.1984, 0.1888, 0.1971, 0.0000],
        [0.1935, 0.1663, 0.1666, 0.1542, 0.1666, 0.1529]],
       grad_fn=<DivBackward0>)


- 虽然我们在技术上已经完成了因果注意力机制的编码，但让我们简要地看一下一种更高效的方法来实现上述功能
- 因此，我们不需要将主对角线以上的注意力权重置零并重新归一化结果，而是在它们进入 softmax 函数之前，将主对角线以上的未归一化注意力分数用负无穷大进行掩码：

<img src="https://sebastianraschka.com/images/LLMs-from-scratch-images/ch03_compressed/21.webp" width="450px">

In [29]:
mask = torch.triu(torch.ones(context_length, context_length), diagonal=1)
masked = attn_scores.masked_fill(mask.bool(), -torch.inf)
print(masked)

tensor([[0.2899,   -inf,   -inf,   -inf,   -inf,   -inf],
        [0.4656, 0.1723,   -inf,   -inf,   -inf,   -inf],
        [0.4594, 0.1703, 0.1731,   -inf,   -inf,   -inf],
        [0.2642, 0.1024, 0.1036, 0.0186,   -inf,   -inf],
        [0.2183, 0.0874, 0.0882, 0.0177, 0.0786,   -inf],
        [0.3408, 0.1270, 0.1290, 0.0198, 0.1290, 0.0078]],
       grad_fn=<MaskedFillBackward0>)


- 如我们所见，现在每行的注意力权重正确地加起来为 1：

In [30]:
attn_weights = torch.softmax(masked / keys.shape[-1]**0.5, dim=-1)
print(attn_weights)

tensor([[1.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.5517, 0.4483, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.3800, 0.3097, 0.3103, 0.0000, 0.0000, 0.0000],
        [0.2758, 0.2460, 0.2462, 0.2319, 0.0000, 0.0000],
        [0.2175, 0.1983, 0.1984, 0.1888, 0.1971, 0.0000],
        [0.1935, 0.1663, 0.1666, 0.1542, 0.1666, 0.1529]],
       grad_fn=<SoftmaxBackward0>)


### 3.5.2 使用 dropout 掩码额外的注意力权重

- 接下来，我们还需要应用 dropout 来减少训练过程中的过拟合
- dropout 可以应用在多个地方：在计算注意力权重之后或者在注意力权重与值向量相乘之后
- 在这里我们将在计算完注意力权重后应用 dropout 掩码，因为这样做更常见

- 此外，在这个特定示例中，我们使用了 50%的 dropout 率，这意味着随机屏蔽掉一半的注意力权重。（当我们稍后训练 GPT 模型时，我们会使用更低的 dropout 率，例如 0.1 或 0.2）

<img src="https://sebastianraschka.com/images/LLMs-from-scratch-images/ch03_compressed/22.webp" width="400px">

- 如果我们应用 0.5（50%）的 dropout 率，未丢弃的值将相应地按 1/0.5=2 的因子进行缩放
- 缩放通过公式 1 / (1 - dropout_rate )计算

In [31]:
torch.manual_seed(123)
dropout = torch.nn.Dropout(0.5) # dropout rate of 50%
example = torch.ones(6, 6) # create a matrix of ones

print(dropout(example))

tensor([[2., 2., 2., 2., 2., 2.],
        [0., 2., 0., 0., 0., 0.],
        [0., 0., 2., 0., 2., 0.],
        [2., 2., 0., 0., 0., 2.],
        [2., 0., 0., 0., 0., 2.],
        [0., 2., 0., 0., 0., 0.]])


In [32]:
torch.manual_seed(123)
print(dropout(attn_weights))

tensor([[2.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.0000, 0.8966, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.0000, 0.0000, 0.6206, 0.0000, 0.0000, 0.0000],
        [0.5517, 0.4921, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.4350, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.0000, 0.3327, 0.0000, 0.0000, 0.0000, 0.0000]],
       grad_fn=<MulBackward0>)


- 请注意由于操作系统不同，生成的 dropout 输出可能会有所不同；你可以在 PyTorch 问题追踪器上了解更多关于这种不一致性的信息

### 3.5.3 实现一个紧凑的因果自注意力类

- 现在我们准备好实现一个可工作的自注意力机制，包括因果掩码和 dropout 掩码
- 还需要实现处理包含多个输入的批次的代码，以便我们的 CausalAttention 类支持第二章中我们实现的数据加载器产生的批次输出
- 为了模拟这种批次输入，我们复制了输入文本示例：

In [33]:
batch = torch.stack((inputs, inputs), dim=0)
print(batch.shape) # 2 inputs with 6 tokens each, and each token has embedding dimension 3

torch.Size([2, 6, 3])


In [34]:
class CausalAttention(nn.Module):

    def __init__(self, d_in, d_out, context_length,
                 dropout, qkv_bias=False):
        super().__init__()
        self.d_out = d_out
        self.W_query = nn.Linear(d_in, d_out, bias=qkv_bias)
        self.W_key   = nn.Linear(d_in, d_out, bias=qkv_bias)
        self.W_value = nn.Linear(d_in, d_out, bias=qkv_bias)
        self.dropout = nn.Dropout(dropout) # New
        self.register_buffer('mask', torch.triu(torch.ones(context_length, context_length), diagonal=1)) # New

    def forward(self, x):
        b, num_tokens, d_in = x.shape # New batch dimension b
        # For inputs where `num_tokens` exceeds `context_length`, this will result in errors
        # in the mask creation further below.
        # In practice, this is not a problem since the LLM (chapters 4-7) ensures that inputs
        # do not exceed `context_length` before reaching this forward method.
        keys = self.W_key(x)
        queries = self.W_query(x)
        values = self.W_value(x)

        attn_scores = queries @ keys.transpose(1, 2) # Changed transpose
        attn_scores.masked_fill_(  # New, _ ops are in-place
            self.mask.bool()[:num_tokens, :num_tokens], -torch.inf)  # `:num_tokens` to account for cases where the number of tokens in the batch is smaller than the supported context_size
        attn_weights = torch.softmax(
            attn_scores / keys.shape[-1]**0.5, dim=-1
        )
        attn_weights = self.dropout(attn_weights) # New

        context_vec = attn_weights @ values
        return context_vec

torch.manual_seed(123)

context_length = batch.shape[1]
ca = CausalAttention(d_in, d_out, context_length, 0.0)

context_vecs = ca(batch)

print(context_vecs)
print("context_vecs.shape:", context_vecs.shape)

tensor([[[-0.4519,  0.2216],
         [-0.5874,  0.0058],
         [-0.6300, -0.0632],
         [-0.5675, -0.0843],
         [-0.5526, -0.0981],
         [-0.5299, -0.1081]],

        [[-0.4519,  0.2216],
         [-0.5874,  0.0058],
         [-0.6300, -0.0632],
         [-0.5675, -0.0843],
         [-0.5526, -0.0981],
         [-0.5299, -0.1081]]], grad_fn=<UnsafeViewBackward0>)
context_vecs.shape: torch.Size([2, 6, 2])


- 请注意，dropout 仅在训练过程中应用，而在推理过程中不应用

<img src="https://sebastianraschka.com/images/LLMs-from-scratch-images/ch03_compressed/23.webp" width="500px">

## 3.6 将单头注意力扩展到多头注意力

### 3.6.1 堆叠多个单头注意力层

- 以下是先前实现的自我注意力的总结（为简洁起见，未显示因果性和 dropout 掩码）

- 这也称为单头注意力：

<img src="https://sebastianraschka.com/images/LLMs-from-scratch-images/ch03_compressed/24.webp" width="400px">

- 我们只需堆叠多个单头注意力模块，即可得到多头注意力模块:

<img src="https://sebastianraschka.com/images/LLMs-from-scratch-images/ch03_compressed/25.webp" width="400px">

- 多头注意力的主要思想是使用不同的、学习得到的线性投影，多次（并行）运行注意力机制。这允许模型在不同位置联合关注来自不同表示子空间的信息。

In [35]:
class MultiHeadAttentionWrapper(nn.Module):

    def __init__(self, d_in, d_out, context_length, dropout, num_heads, qkv_bias=False):
        super().__init__()
        self.heads = nn.ModuleList(
            [CausalAttention(d_in, d_out, context_length, dropout, qkv_bias)
             for _ in range(num_heads)]
        )

    def forward(self, x):
        return torch.cat([head(x) for head in self.heads], dim=-1)


torch.manual_seed(123)

context_length = batch.shape[1] # This is the number of tokens
d_in, d_out = 3, 2
mha = MultiHeadAttentionWrapper(
    d_in, d_out, context_length, 0.0, num_heads=2
)

context_vecs = mha(batch)

print(context_vecs)
print("context_vecs.shape:", context_vecs.shape)

tensor([[[-0.4519,  0.2216,  0.4772,  0.1063],
         [-0.5874,  0.0058,  0.5891,  0.3257],
         [-0.6300, -0.0632,  0.6202,  0.3860],
         [-0.5675, -0.0843,  0.5478,  0.3589],
         [-0.5526, -0.0981,  0.5321,  0.3428],
         [-0.5299, -0.1081,  0.5077,  0.3493]],

        [[-0.4519,  0.2216,  0.4772,  0.1063],
         [-0.5874,  0.0058,  0.5891,  0.3257],
         [-0.6300, -0.0632,  0.6202,  0.3860],
         [-0.5675, -0.0843,  0.5478,  0.3589],
         [-0.5526, -0.0981,  0.5321,  0.3428],
         [-0.5299, -0.1081,  0.5077,  0.3493]]], grad_fn=<CatBackward0>)
context_vecs.shape: torch.Size([2, 6, 4])


- 在上述实现中，嵌入维度为 4，因为我们把键向量、查询向量、值向量和上下文向量都视为嵌入维度。而且因为我们有 2 个注意力头，所以输出嵌入维度为 2*2=4

### 3.6.2 使用权重拆分实现多头注意力

- 虽然以上是多头注意力机制的一个直观且完全功能的实现（包装了之前单头注意力 CausalAttention 的实现），但我们可以编写一个名为 MultiHeadAttention 的独立类来实现相同的功能

- 对于这个独立的 MultiHeadAttention 类，我们不连接单个注意力头
- 相反，我们创建单个 W_query、W_key 和 W_value 权重矩阵，然后将这些矩阵分割成每个注意力头的独立矩阵：

In [36]:
class MultiHeadAttention(nn.Module):
    def __init__(self, d_in, d_out, context_length, dropout, num_heads, qkv_bias=False):
        super().__init__()
        assert (d_out % num_heads == 0), \
            "d_out must be divisible by num_heads"

        self.d_out = d_out
        self.num_heads = num_heads
        self.head_dim = d_out // num_heads # Reduce the projection dim to match desired output dim

        self.W_query = nn.Linear(d_in, d_out, bias=qkv_bias)
        self.W_key = nn.Linear(d_in, d_out, bias=qkv_bias)
        self.W_value = nn.Linear(d_in, d_out, bias=qkv_bias)
        self.out_proj = nn.Linear(d_out, d_out)  # Linear layer to combine head outputs
        self.dropout = nn.Dropout(dropout)
        self.register_buffer(
            "mask",
            torch.triu(torch.ones(context_length, context_length),
                       diagonal=1)
        )

    def forward(self, x):
        b, num_tokens, d_in = x.shape
        # As in `CausalAttention`, for inputs where `num_tokens` exceeds `context_length`,
        # this will result in errors in the mask creation further below.
        # In practice, this is not a problem since the LLM (chapters 4-7) ensures that inputs
        # do not exceed `context_length` before reaching this forwar

        keys = self.W_key(x) # Shape: (b, num_tokens, d_out)
        queries = self.W_query(x)
        values = self.W_value(x)

        # We implicitly split the matrix by adding a `num_heads` dimension
        # Unroll last dim: (b, num_tokens, d_out) -> (b, num_tokens, num_heads, head_dim)
        keys = keys.view(b, num_tokens, self.num_heads, self.head_dim)
        values = values.view(b, num_tokens, self.num_heads, self.head_dim)
        queries = queries.view(b, num_tokens, self.num_heads, self.head_dim)

        # Transpose: (b, num_tokens, num_heads, head_dim) -> (b, num_heads, num_tokens, head_dim)
        keys = keys.transpose(1, 2)
        queries = queries.transpose(1, 2)
        values = values.transpose(1, 2)

        # Compute scaled dot-product attention (aka self-attention) with a causal mask
        attn_scores = queries @ keys.transpose(2, 3)  # Dot product for each head

        # Original mask truncated to the number of tokens and converted to boolean
        mask_bool = self.mask.bool()[:num_tokens, :num_tokens]

        # Use the mask to fill attention scores
        attn_scores.masked_fill_(mask_bool, -torch.inf)

        attn_weights = torch.softmax(attn_scores / keys.shape[-1]**0.5, dim=-1)
        attn_weights = self.dropout(attn_weights)

        # Shape: (b, num_tokens, num_heads, head_dim)
        context_vec = (attn_weights @ values).transpose(1, 2)

        # Combine heads, where self.d_out = self.num_heads * self.head_dim
        context_vec = context_vec.contiguous().view(b, num_tokens, self.d_out)
        context_vec = self.out_proj(context_vec) # optional projection

        return context_vec

torch.manual_seed(123)

batch_size, context_length, d_in = batch.shape
d_out = 2
mha = MultiHeadAttention(d_in, d_out, context_length, 0.0, num_heads=2)

context_vecs = mha(batch)

print(context_vecs)
print("context_vecs.shape:", context_vecs.shape)

tensor([[[0.3190, 0.4858],
         [0.2943, 0.3897],
         [0.2856, 0.3593],
         [0.2693, 0.3873],
         [0.2639, 0.3928],
         [0.2575, 0.4028]],

        [[0.3190, 0.4858],
         [0.2943, 0.3897],
         [0.2856, 0.3593],
         [0.2693, 0.3873],
         [0.2639, 0.3928],
         [0.2575, 0.4028]]], grad_fn=<ViewBackward0>)
context_vecs.shape: torch.Size([2, 6, 2])


- 请注意，以上本质上是对 MultiHeadAttentionWrapper 的一个重写版本，它更高效
- 生成的输出看起来略有不同，因为随机权重初始化不同，但两者都是完全可用的实现，可以在接下来的章节中我们将实现的 GPT 类中使用
- 请注意，此外，我们在上面的 MultiHeadAttention 类中添加了一个线性投影层（ self.out_proj ）。这只是一个不改变维度的线性变换。在 LLM 实现中使用这种投影层是一种标准惯例，但并非严格必要（最近的研究表明，可以将其移除而不影响建模性能；参见本章末尾的进一步阅读部分）


<img src="https://sebastianraschka.com/images/LLMs-from-scratch-images/ch03_compressed/26.webp" width="400px">

- 注意，如果你对上述的紧凑且高效的实现感兴趣，也可以考虑 PyTorch 中的 torch.nn.MultiheadAttention 类

- 由于上述实现乍一看可能有点复杂，让我们看看执行 attn_scores = queries @ keys.transpose(2, 3) 时会发生什么：

In [37]:
# (b, num_heads, num_tokens, head_dim) = (1, 2, 3, 4)
a = torch.tensor([[[[0.2745, 0.6584, 0.2775, 0.8573],
                    [0.8993, 0.0390, 0.9268, 0.7388],
                    [0.7179, 0.7058, 0.9156, 0.4340]],

                   [[0.0772, 0.3565, 0.1479, 0.5331],
                    [0.4066, 0.2318, 0.4545, 0.9737],
                    [0.4606, 0.5159, 0.4220, 0.5786]]]])

print(a @ a.transpose(2, 3))

tensor([[[[1.3208, 1.1631, 1.2879],
          [1.1631, 2.2150, 1.8424],
          [1.2879, 1.8424, 2.0402]],

         [[0.4391, 0.7003, 0.5903],
          [0.7003, 1.3737, 1.0620],
          [0.5903, 1.0620, 0.9912]]]])


- 在这种情况下，PyTorch 中的矩阵乘法实现将处理 4 维输入张量，以便在最后两个维度（num_tokens, head_dim）之间进行矩阵乘法，然后为每个头重复执行

- 例如，以下成为分别计算每个头的矩阵乘法的一种更紧凑的方式：

In [38]:
first_head = a[0, 0, :, :]
first_res = first_head @ first_head.T
print("First head:\n", first_res)

second_head = a[0, 1, :, :]
second_res = second_head @ second_head.T
print("\nSecond head:\n", second_res)

First head:
 tensor([[1.3208, 1.1631, 1.2879],
        [1.1631, 2.2150, 1.8424],
        [1.2879, 1.8424, 2.0402]])

Second head:
 tensor([[0.4391, 0.7003, 0.5903],
        [0.7003, 1.3737, 1.0620],
        [0.5903, 1.0620, 0.9912]])


# Summary and takeaways

- 查看 ./multihead-attention.ipynb 代码笔记本，它是一个简洁版的数据加载器（第 2 章）以及我们在本章实现的多头注意力类，我们将在接下来的章节中需要它来训练 GPT 模型。
- 你可以在 ./exercise-solutions.ipynb 中找到练习题的解答。